In [408]:
import pandas as pd
import numpy as np
import os



In [409]:
def load_and_prepare_data(year, file_prefix, columns, offense=True):
    file_suffix = 'OFF' if offense else 'DEF'
    file_path = f'{file_prefix}{year}{file_suffix}.csv'
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        return pd.DataFrame()  
    data = pd.read_csv(file_path)
    data['Year'] = year
    data = data[1:33]
    data.columns = columns
    data['Year'] = data['Year'].astype('int')
    return data



In [410]:
offensive_columns = [
    'O_Rank', 'Team', 'O_Games_Played', 'O_Points_For', 'O_Total_Yards',
    'O_Plays', 'O_Y/Play', 'O_TO', 'O_Fumbles_Lost', 'O_1st_D', 'O_P_Completions',
    'O_P_Attempts', 'O_P_Yards', 'O_P_TD', 'O_P_Int', 'O_P_Y/PA', 'O_P_Passing_1st_D',
    'O_R_Att', 'O_R_Yards', 'O_R_TD', 'O_R_Y/A', 'O_R_1st_D', 'O_Pe', 'O_Pe_Yards',
    'O_Pe_1st_D', 'O_Scoring_Drives', 'O_TO_%', 'O_Expected_Points', 'Year'
]

In [411]:
defensive_columns = [
    'D_Rank', 'Team', 'D_Games_Played', 'D_Points_Allowed', 'D_Total_Yards',
    'D_Plays', 'D_Y/Play', 'D_TD', 'D_Fumbles_Lost', 'D_1st_D', 'D_P_Completions',
    'D_P_Attempts', 'D_P_Yards', 'D_P_TD', 'D_P_Int', 'D_P_Y/PA', 'D_P_Passing_1st_D',
    'D_R_Att', 'D_R_Yards', 'D_R_TD', 'D_R_Y/A', 'D_R_1st_D', 'D_Pe', 'D_Pe_Yards',
    'D_Pe_1st_D', 'D_Scoring_Drives', 'D_TD_%', 'D_Expected_Points', 'Year'
]

years = [2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014]

In [412]:
offensive_data = [load_and_prepare_data(year, 'NFL-', offensive_columns, offense=True) for year in years]
defensive_data = [load_and_prepare_data(year, 'NFL-', defensive_columns, offense=False) for year in years]

OFF_total = pd.concat(offensive_data)
DEF_total = pd.concat(defensive_data)

In [413]:
# Merge offensive and defensive data
combined_data = OFF_total.merge(DEF_total, on=['Team', 'Year'])

In [414]:
# Load and prepare the 2024 data(2024 current ranks for offense and defense)
OFF_2024 = load_and_prepare_data(2024, 'NFL-', offensive_columns, offense=True)
DEF_2024 = load_and_prepare_data(2024, 'NFL-', defensive_columns, offense=False)

testing_set = OFF_2024.merge(DEF_2024, on=['Team', 'Year'])
testing_set.to_csv(r'C:\Users\richa\ProjectNFL\testingset.csv', index=False)

In [415]:
# Combine the testing set with the combined data
mydata = pd.concat([testing_set, combined_data])
mydata.head()

,O_Rank,Team,O_Games_Played,O_Points_For,O_Total_Yards,O_Plays,O_Y/Play,O_TO,O_Fumbles_Lost,O_1st_D,O_P_Completions,O_P_Attempts,O_P_Yards,O_P_TD,O_P_Int,O_P_Y/PA,O_P_Passing_1st_D,O_R_Att,O_R_Yards,O_R_TD,O_R_Y/A,O_R_1st_D,O_Pe,O_Pe_Yards,O_Pe_1st_D,O_Scoring_Drives,O_TO_%,O_Expected_Points,Year,D_Rank,D_Games_Played,D_Points_Allowed,D_Total_Yards,D_Plays,D_Y/Play,D_TD,D_Fumbles_Lost,D_1st_D,D_P_Completions,D_P_Attempts,D_P_Yards,D_P_TD,D_P_Int,D_P_Y/PA,D_P_Passing_1st_D,D_R_Att,D_R_Yards,D_R_TD,D_R_Y/A,D_R_1st_D,D_Pe,D_Pe_Yards,D_Pe_1st_D,D_Scoring_Drives,D_TD_%,D_Expected_Points
0,1,Detroit Lions,15,493,6129,983,6.2,13,3,363,346,484,3971,35,10,7.7,212,470,2158,24,4.6,128,93,882,23,50.9,8.1,223.40,2024,8,15,299,5083,894,5.7,22,8,288,325,529,3606,15,14,6.4,172,332,1477,16,4.4,76,103,802,40,34.8,13.7,-18.71
1,2,Buffalo Bills,15,469,5498,898,6.1,8,2,321,294,460,3495,26,6,7.4,164,424,2003,29,4.7,125,99,738,32,50.7,5.3,213.51,2024,11,15,331,5216,934,5.6,28,14,317,351,516,3442,25,14,6.3,194,384,1774,12,4.6,95,104,837,28,36.4,17.2,-61.87
2,3,Baltimore Ravens,15,452,6355,927,6.9,11,7,348,291,428,3637,37,4,8.1,179,476,2718,17,5.7,140,119,1007,29,47.5,6.9,230.15,2024,19,15,349,5070,953,5.3,14,5,307,360,559,3823,26,9,6.3,201,347,1247,12,3.6,67,89,690,39,40.5,8.6,-67.32
3,4,Washington Commanders,15,432,5603,969,5.8,14,6,333,320,458,3313,24,8,6.7,168,472,2290,23,4.9,134,93,797,31,50.6,8.3,186.94,2024,18,15,348,4843,894,5.4,16,10,289,268,427,2782,24,6,6.0,152,428,2061,15,4.8,105,90,757,32,40.8,9.2,-78.82
4,5,Tampa Bay Buccaneers,15,427,5847,956,6.1,22,7,342,360,507,3692,34,15,6.8,196,412,2155,15,5.2,117,91,709,29,46.3,13.6,148.92,2024,22,15,352,5314,956,5.6,18,11,304,377,560,3754,24,7,6.3,188,357,1560,13,4.4,88,84,790,28,40.5,9.8,-77.40


In [416]:
# Convert columns to float
total_columns = [
    'O_Games_Played', 'O_Points_For', 'O_Total_Yards', 'O_Plays', 'O_Y/Play',
    'O_TO', 'O_Fumbles_Lost', 'O_1st_D', 'O_P_Completions', 'O_P_Attempts',
    'O_P_Yards', 'O_P_TD', 'O_P_Int', 'O_P_Y/PA', 'O_P_Passing_1st_D', 'O_R_Att',
    'O_R_Yards', 'O_R_TD', 'O_R_Y/A', 'O_R_1st_D', 'O_Pe', 'O_Pe_Yards',
    'O_Pe_1st_D', 'O_Scoring_Drives', 'O_TO_%', 'O_Expected_Points',
    'D_Games_Played', 'D_Points_Allowed', 'D_Total_Yards', 'D_Plays', 'D_Y/Play',
    'D_TD', 'D_Fumbles_Lost', 'D_1st_D', 'D_P_Completions', 'D_P_Attempts',
    'D_P_Yards', 'D_P_TD', 'D_P_Int', 'D_P_Y/PA', 'D_P_Passing_1st_D', 'D_R_Att',
    'D_R_Yards', 'D_R_TD', 'D_R_Y/A', 'D_R_1st_D', 'D_Pe', 'D_Pe_Yards',
    'D_Pe_1st_D', 'D_Scoring_Drives', 'D_TD_%', 'D_Expected_Points'
]

mydata[total_columns] = mydata[total_columns].astype('float')

In [417]:
mydata['Superbowl'] = 0
superbowl = pd.read_csv('superbowl.csv')
superbowl.head()

,Date,SB,Winner,Pts,Loser,Pts.1,MVP,Stadium,City,State
0,Feb 11 2024,LVIII (58),Kansas City Chiefs,25,San Francisco 49ers,22,Patrick Mahomes,Allegiant Stadium,Las Vegas,Nevada
1,Feb 12 2023,LVII (57),Kansas City Chiefs,38,Philadelphia Eagles,35,Patrick Mahomes,State Farm Stadium,Glendale,Arizona
2,Feb 13 2022,LVI (56),Los Angeles Rams,23,Cincinnati Bengals,20,Cooper Kupp,SoFi Stadium,Inglewood,California
3,Feb 7 2021,LV (55),Tampa Bay Buccaneers,31,Kansas City Chiefs,9,Tom Brady,Raymond James Stadium,Tampa,Florida
4,Feb 2 2020,LIV (54),Kansas City Chiefs,31,San Francisco 49ers,20,Patrick Mahomes,Hard Rock Stadium,Miami Gardens,Florida


In [418]:
superbowl['Date'] = pd.to_datetime(superbowl['Date'])

In [419]:
superbowl['Year'] = pd.DatetimeIndex(superbowl['Date']).year
superbowl= superbowl[['Year', 'Winner']]
superbowl.head()

,Year,Winner
0,2024,Kansas City Chiefs
1,2023,Kansas City Chiefs
2,2022,Los Angeles Rams
3,2021,Tampa Bay Buccaneers
4,2020,Kansas City Chiefs


In [420]:
superbowl['Year'] = superbowl['Year'] - 1

In [421]:
superbowl.Year = superbowl.Year.astype('object')

In [422]:
superbowl['Superbowl'] = 1

In [423]:
superbowl.Year = superbowl.Year.astype('int')
mydata.Year = mydata.Year.astype('int')

test = pd.merge(mydata, superbowl, left_on = ['Year','Team'], right_on = ['Year', 'Winner'], how = 'left')
test['Superbowl_y'].value_counts()

Superbowl_y
1.0    6
Name: count, dtype: int64

In [424]:
test.drop(['Superbowl_x', 'Winner'], axis=1, inplace = True)

In [425]:
pd.set_option('display.max_rows', 500)
test.Superbowl_y = test.Superbowl_y.fillna(0)
test.Superbowl_y = test.Superbowl_y.astype('int')
test.rename(columns = {'Superbowl_y': 'Superbowl'}, inplace = True)

In [426]:
test.dtypes
test.Superbowl.sum()

np.int64(6)

In [427]:
# Import necessary libraries
import sys
import pylab as pl
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt


In [428]:
test

,O_Rank,Team,O_Games_Played,O_Points_For,O_Total_Yards,O_Plays,O_Y/Play,O_TO,O_Fumbles_Lost,O_1st_D,O_P_Completions,O_P_Attempts,O_P_Yards,O_P_TD,O_P_Int,O_P_Y/PA,O_P_Passing_1st_D,O_R_Att,O_R_Yards,O_R_TD,O_R_Y/A,O_R_1st_D,O_Pe,O_Pe_Yards,O_Pe_1st_D,O_Scoring_Drives,O_TO_%,O_Expected_Points,Year,D_Rank,D_Games_Played,D_Points_Allowed,D_Total_Yards,D_Plays,D_Y/Play,D_TD,D_Fumbles_Lost,D_1st_D,D_P_Completions,D_P_Attempts,D_P_Yards,D_P_TD,D_P_Int,D_P_Y/PA,D_P_Passing_1st_D,D_R_Att,D_R_Yards,D_R_TD,D_R_Y/A,D_R_1st_D,D_Pe,D_Pe_Yards,D_Pe_1st_D,D_Scoring_Drives,D_TD_%,D_Expected_Points,Superbowl
0,1,Detroit Lions,15.0,493.0,6129.0,983.0,6.2,13.0,3.0,363.0,346.0,484.0,3971.0,35.0,10.0,7.7,212.0,470.0,2158.0,24.0,4.6,128.0,93.0,882.0,23.0,50.9,8.1,223.40,2024,8,15.0,299.0,5083.0,894.0,5.7,22.0,8.0,288.0,325.0,529.0,3606.0,15.0,14.0,6.4,172.0,332.0,1477.0,16.0,4.4,76.0,103.0,802.0,40.0,34.8,13.7,-18.71,0
1,2,Buffalo Bills,15.0,469.0,5498.0,898.0,6.1,8.0,2.0,321.0,294.0,460.0,3495.0,26.0,6.0,7.4,164.0,424.0,2003.0,29.0,4.7,125.0,99.0,738.0,32.0,50.7,5.3,213.51,2024,11,15.0,331.0,5216.0,934.0,5.6,28.0,14.0,317.0,351.0,516.0,3442.0,25.0,14.0,6.3,194.0,384.0,1774.0,12.0,4.6,95.0,104.0,837.0,28.0,36.4,17.2,-61.87,0
2,3,Baltimore Ravens,15.0,452.0,6355.0,927.0,6.9,11.0,7.0,348.0,291.0,428.0,3637.0,37.0,4.0,8.1,179.0,476.0,2718.0,17.0,5.7,140.0,119.0,1007.0,29.0,47.5,6.9,230.15,2024,19,15.0,349.0,5070.0,953.0,5.3,14.0,5.0,307.0,360.0,559.0,3823.0,26.0,9.0,6.3,201.0,347.0,1247.0,12.0,3.6,67.0,89.0,690.0,39.0,40.5,8.6,-67.32,0
3,4,Washington Commanders,15.0,432.0,5603.0,969.0,5.8,14.0,6.0,333.0,320.0,458.0,3313.0,24.0,8.0,6.7,168.0,472.0,2290.0,23.0,4.9,134.0,93.0,797.0,31.0,50.6,8.3,186.94,2024,18,15.0,348.0,4843.0,894.0,5.4,16.0,10.0,289.0,268.0,427.0,2782.0,24.0,6.0,6.0,152.0,428.0,2061.0,15.0,4.8,105.0,90.0,757.0,32.0,40.8,9.2,-78.82,0
4,5,Tampa Bay Buccaneers,15.0,427.0,5847.0,956.0,6.1,22.0,7.0,342.0,360.0,507.0,3692.0,34.0,15.0,6.8,196.0,412.0,2155.0,15.0,5.2,117.0,91.0,709.0,29.0,46.3,13.6,148.92,2024,22,15.0,352.0,5314.0,956.0,5.6,18.0,11.0,304.0,377.0,560.0,3754.0,24.0,7.0,6.3,188.0,357.0,1560.0,13.0,4.4,88.0,84.0,790.0,28.0,40.5,9.8,-77.40,0
5,6,Cincinnati Bengals,15.0,423.0,5401.0,925.0,5.8,19.0,11.0,321.0,384.0,557.0,4010.0,39.0,8.0,6.8,215.0,331.0,1391.0,10.0,4.2,77.0,92.0,740.0,29.0,43.4,10.7,134.55,2024,28,15.0,393.0,5399.0,970.0,5.6,23.0,9.0,326.0,331.0,511.0,3474.0,26.0,14.0,6.4,174.0,429.0,1925.0,19.0,4.5,122.0,91.0,771.0,30.0,40.1,13.6,-102.15,0
6,7,Green Bay Packers,15.0,413.0,5665.0,909.0,6.2,16.0,5.0,306.0,272.0,424.0,3456.0,27.0,11.0,7.9,153.0,469.0,2209.0,19.0,4.7,125.0,100.0,740.0,28.0,46.5,9.4,142.89,2024,6,15.0,287.0,4682.0,909.0,5.2,28.0,13.0,285.0,327.0,492.0,3145.0,19.0,15.0,5.9,170.0,374.0,1537.0,12.0,4.1,95.0,100.0,848.0,20.0,32.5,16.6,19.40,0
7,8,Philadelphia Eagles,15.0,402.0,5563.0,985.0,5.6,15.0,9.0,319.0,263.0,388.0,2745.0,19.0,6.0,6.4,147.0,554.0,2818.0,28.0,5.1,150.0,98.0,768.0,22.0,43.8,7.4,131.73,2024,5,15.0,283.0,4226.0,885.0,4.8,21.0,11.0,267.0,299.0,484.0,2682.0,20.0,10.0,5.1,149.0,361.0,1544.0,9.0,4.3,94.0,71.0,585.0,24.0,32.9,12.7,25.94,0
8,9,Minnesota Vikings,15.0,396.0,5195.0,922.0,5.6,20.0,9.0,313.0,313.0,464.0,3529.0,32.0,11.0,6.9,183.0,412.0,1666.0,9.0,4.0,93.0,97.0,768.0,37.0,42.0,11.2,92.20,2024,2,15.0,276.0,5037.0,955.0,5.3,30.0,8.0,284.0,369.0,574.0,3731.0,22.0,22.0,6.0,193.0,337.0,1306.0,7.0,3.9,70.0,126.0,1085.0,21.0,30.1,16.9,48.99,0
9,10,Denver Broncos,15.0,363.0,4710.0,923.0,5.1,18.0,7.0,267.0,329.0,510.0,3083.0,23.0,11.0,5.8,147.0,391.0,1627.0,11.0,4.2,101.0,96.0,865.0,19.0,36.6,10.5,10.25,2024,4,15.0,281.0,4794.0,976.0,4.9,24.0,9.0,288.0,344.0,540.0,3297.0,19.0,15.0,5.6,176.0,385.0,1497.0,9.0,3.9,76.0,95.0,693.0,36.0,32.2,13.2,63.47,0


In [429]:
# Set print options for numpy
np.set_printoptions(threshold=sys.maxsize)


In [430]:
# Convert ranks to integers
test.O_Rank = test.O_Rank.astype(int)
test.D_Rank = test.D_Rank.astype(int)

# Define features for training columns
training_columns = [
    'O_Rank', 'O_Points_For', 'O_Total_Yards', 'O_Y/Play', 'O_TO', 'O_1st_D',
    'O_P_Completions', 'O_P_Yards', 'O_P_TD', 'O_P_Int', 'O_P_Y/PA', 'O_P_Passing_1st_D',
    'O_R_Yards', 'O_R_TD', 'O_R_Y/A', 'O_R_1st_D', 'O_Pe', 'O_Pe_Yards',
    'O_Scoring_Drives', 'D_Rank', 'D_Points_Allowed', 'D_Total_Yards',
    'D_Y/Play', 'D_TD', 'D_Fumbles_Lost', 'D_1st_D', 'D_P_Completions',
    'D_P_Yards', 'D_P_TD', 'D_P_Int', 'D_P_Y/PA', 'D_P_Passing_1st_D',
    'D_R_Yards', 'D_R_TD', 'D_R_Y/A', 'D_R_1st_D', 'D_Pe', 'D_Pe_Yards',
    'D_Scoring_Drives'
]

# Prepare data
X = test[training_columns].values
y = test['Superbowl'].values

In [431]:
from sklearn import preprocessing
# Standardize the training columns
X = preprocessing.StandardScaler().fit_transform(X)

In [432]:
# Select training and testing data
teams = test.loc[ :31 , 'Team']

O_rank = test.loc[0:31, 'O_Rank']
D_rank = test.loc[0:31, 'D_Rank']

X_test = X[0:32]
X_train = X[32:]

y_test = y[0:32]
y_train = y[32:]

print(X_test.shape)
print(y_test.shape)
print(X_train.shape)
print(y_train.shape)



(32, 39)
(32,)
(320, 39)
(320,)


In [433]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
predicts = LR.predict(X_test)
predicts

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [434]:
from sklearn import metrics

# Evaluate Logistic Regression Model
predicts_binary = LR.predict(X_train)
print("Logistic Regression Accuracy:", metrics.accuracy_score(y_train, predicts_binary))

Logistic Regression Accuracy: 0.98125


In [435]:
# Predictions DataFrame for Logistic Regression
predictions = pd.DataFrame({'Team': teams, 'Predictions': predicts})
predictions

,Team,Predictions
0,Detroit Lions,0
1,Buffalo Bills,0
2,Baltimore Ravens,0
3,Washington Commanders,0
4,Tampa Bay Buccaneers,0
5,Cincinnati Bengals,0
6,Green Bay Packers,0
7,Philadelphia Eagles,0
8,Minnesota Vikings,0
9,Denver Broncos,0


In [436]:
predicts_prob = LR.predict_proba(X_test)
predicts_prob

array([[0.59491911, 0.40508089],
       [0.59985444, 0.40014556],
       [0.60099549, 0.39900451],
       [0.52631905, 0.47368095],
       [0.59482328, 0.40517672],
       [0.58548771, 0.41451229],
       [0.58681725, 0.41318275],
       [0.5453807 , 0.4546193 ],
       [0.6712994 , 0.3287006 ],
       [0.6096775 , 0.3903225 ],
       [0.57226419, 0.42773581],
       [0.63234911, 0.36765089],
       [0.59817977, 0.40182023],
       [0.53808435, 0.46191565],
       [0.64393766, 0.35606234],
       [0.4934847 , 0.5065153 ],
       [0.5868472 , 0.4131528 ],
       [0.53315628, 0.46684372],
       [0.60672395, 0.39327605],
       [0.64440029, 0.35559971],
       [0.62616018, 0.37383982],
       [0.62654663, 0.37345337],
       [0.58460527, 0.41539473],
       [0.57065966, 0.42934034],
       [0.6236539 , 0.3763461 ],
       [0.62307438, 0.37692562],
       [0.65585903, 0.34414097],
       [0.62567955, 0.37432045],
       [0.58854547, 0.41145453],
       [0.60080219, 0.39919781],
       [0.

In [437]:
# Prepare predictions DataFrame with probabilities
predictions2 = pd.DataFrame({
    'Team': teams,
    'Prediction': predicts_prob[:, 1],
    'O_Rank': O_rank,
    'D_Rank': D_rank
}).sort_values(by='Prediction', ascending=False)
predictions2['Prediction_Rank'] = predictions2['Prediction'].rank(ascending=False)

In [438]:
# AdaBoost with SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.svm import SVC
from sklearn import metrics

svc = SVC(probability=True, kernel='linear')
abc = AdaBoostClassifier(n_estimators=20, estimator=svc, learning_rate=1)
model = abc.fit(X_train, y_train)

In [439]:
# AdaBoost Predictions
y_pred_binary = model.predict(X_train)
y_pred = model.predict_proba(X_test)

In [440]:
# Prepare predictions DataFrame for AdaBoost
predictions3 = pd.DataFrame({
    'Team': teams,
    'Prediction': y_pred[:, 1],
    'O_Rank': O_rank,
    'D_Rank': D_rank
}).sort_values(by='Prediction', ascending=False)
predictions3['Prediction_Rank'] = predictions3['Prediction'].rank(ascending=False)

In [441]:
predictions3

,Team,Prediction,O_Rank,D_Rank,Prediction_Rank
17,Los Angeles Chargers,0.296987,18,1,1.0
8,Minnesota Vikings,0.279584,9,2,2.0
14,Seattle Seahawks,0.244986,15,12,4.5
12,Houston Texans,0.244986,13,9,4.5
18,Atlanta Falcons,0.244986,19,20,4.5
28,Las Vegas Raiders,0.244986,29,27,4.5
10,Kansas City Chiefs,0.229635,11,3,8.0
9,Denver Broncos,0.229635,10,4,8.0
22,Miami Dolphins,0.229635,23,10,8.0
0,Detroit Lions,0.223399,1,8,10.0


In [442]:
# Calculate Average and New Rank
df = predictions3.copy()
df['Average_Rank'] = (df['O_Rank'] + df['D_Rank']) / 2
df['New_Rank'] = df['Average_Rank'].rank(method='dense')
df = df.sort_values(by='New_Rank').reset_index(drop=True)
df

,Team,Prediction,O_Rank,D_Rank,Prediction_Rank,Average_Rank,New_Rank
0,Detroit Lions,0.223399,1,8,10.0,4.5,1.0
1,Minnesota Vikings,0.279584,9,2,2.0,5.5,2.0
2,Buffalo Bills,0.215256,2,11,11.0,6.5,3.0
3,Green Bay Packers,0.210617,7,6,12.0,6.5,3.0
4,Philadelphia Eagles,0.173068,8,5,21.0,6.5,3.0
5,Kansas City Chiefs,0.229635,11,3,8.0,7.0,4.0
6,Denver Broncos,0.229635,10,4,8.0,7.0,4.0
7,Los Angeles Chargers,0.296987,18,1,1.0,9.5,5.0
8,Pittsburgh Steelers,0.161262,12,7,25.5,9.5,5.0
9,Baltimore Ravens,0.131058,3,19,31.0,11.0,6.0


In [443]:
# Print final DataFrame
print(df[['Team', 'O_Rank', 'D_Rank', 'Average_Rank', 'New_Rank']])

                     Team  O_Rank  D_Rank  Average_Rank  New_Rank
0           Detroit Lions       1       8           4.5       1.0
1       Minnesota Vikings       9       2           5.5       2.0
2           Buffalo Bills       2      11           6.5       3.0
3       Green Bay Packers       7       6           6.5       3.0
4     Philadelphia Eagles       8       5           6.5       3.0
5      Kansas City Chiefs      11       3           7.0       4.0
6          Denver Broncos      10       4           7.0       4.0
7    Los Angeles Chargers      18       1           9.5       5.0
8     Pittsburgh Steelers      12       7           9.5       5.0
9        Baltimore Ravens       3      19          11.0       6.0
10  Washington Commanders       4      18          11.0       6.0
11         Houston Texans      13       9          11.0       6.0
12   Tampa Bay Buccaneers       5      22          13.5       7.0
13      Arizona Cardinals      14      13          13.5       7.0
14       S

In [444]:
import pandas as pd

# Sample DataFrame (as per your data)
data = {
    'Team': ['Detroit Lions', 'Buffalo Bills', 'Minnesota Vikings', 'Green Bay Packers', 
             'Denver Broncos', 'Kansas City Chiefs', 'Philadelphia Eagles', 'Baltimore Ravens',
             'Pittsburgh Steelers', 'Los Angeles Chargers', 'Washington Commanders', 
             'Houston Texans', 'Tampa Bay Buccaneers', 'Seattle Seahawks', 
             'Arizona Cardinals', 'Cincinnati Bengals', 'Los Angeles Rams', 
             'Miami Dolphins', 'San Francisco 49ers', 'New Orleans Saints', 
             'New York Jets', 'Atlanta Falcons', 'Chicago Bears', 
             'Indianapolis Colts', 'Dallas Cowboys', 'Tennessee Titans', 
             'New England Patriots', 'Las Vegas Raiders', 'New York Giants', 
             'Jacksonville Jaguars', 'Cleveland Browns', 'Carolina Panthers'],
    'O_Rank': [1, 2, 9, 7, 10, 11, 8, 3, 12, 18, 4, 13, 5, 15, 14, 6, 
               17, 23, 16, 22, 24, 19, 26, 21, 20, 25, 30, 29, 32, 28, 
               31, 27],
    'D_Rank': [8, 11, 2, 6, 4, 3, 5, 19, 7, 1, 18, 9, 22, 12, 13, 28, 
               17, 10, 21, 16, 15, 20, 14, 23, 30, 31, 24, 27, 25, 29, 
               26, 32],
    'Losses': [2, 3, 2, 4, 6, 1, 3, 5, 5, 6, 5, 6, 7, 7, 8, 8, 6, 8,
               9, 10, 11, 7, 11, 8, 8, 12, 12, 12, 13, 12, 12, 12]  
    }

In [445]:
# Calculate Average Rank including Losses
df = pd.DataFrame(data)
df['Average_Rank'] = (df['O_Rank'] + df['D_Rank'] + df['Losses']) / 3
df['New_Rank'] = df['Average_Rank'].rank(method='dense')
df = df.sort_values(by='New_Rank').reset_index(drop=True)

In [446]:
# Print final DataFrame
print(df[['Team', 'O_Rank', 'D_Rank', 'Losses', 'Average_Rank', 'New_Rank']])

                     Team  O_Rank  D_Rank  Losses  Average_Rank  New_Rank
0           Detroit Lions       1       8       2      3.666667       1.0
1       Minnesota Vikings       9       2       2      4.333333       2.0
2      Kansas City Chiefs      11       3       1      5.000000       3.0
3           Buffalo Bills       2      11       3      5.333333       4.0
4     Philadelphia Eagles       8       5       3      5.333333       4.0
5       Green Bay Packers       7       6       4      5.666667       5.0
6          Denver Broncos      10       4       6      6.666667       6.0
7     Pittsburgh Steelers      12       7       5      8.000000       7.0
8    Los Angeles Chargers      18       1       6      8.333333       8.0
9        Baltimore Ravens       3      19       5      9.000000       9.0
10  Washington Commanders       4      18       5      9.000000       9.0
11         Houston Texans      13       9       6      9.333333      10.0
12   Tampa Bay Buccaneers       5     

In [447]:
df_sorted = df.sort_values(by=['New_Rank', 'O_Rank'])
print(df_sorted)

                     Team  O_Rank  D_Rank  Losses  Average_Rank  New_Rank
0           Detroit Lions       1       8       2      3.666667       1.0
1       Minnesota Vikings       9       2       2      4.333333       2.0
2      Kansas City Chiefs      11       3       1      5.000000       3.0
3           Buffalo Bills       2      11       3      5.333333       4.0
4     Philadelphia Eagles       8       5       3      5.333333       4.0
5       Green Bay Packers       7       6       4      5.666667       5.0
6          Denver Broncos      10       4       6      6.666667       6.0
7     Pittsburgh Steelers      12       7       5      8.000000       7.0
8    Los Angeles Chargers      18       1       6      8.333333       8.0
9        Baltimore Ravens       3      19       5      9.000000       9.0
10  Washington Commanders       4      18       5      9.000000       9.0
11         Houston Texans      13       9       6      9.333333      10.0
12   Tampa Bay Buccaneers       5     

In [448]:
df_sorted.to_csv('updated_ranks.csv', index = False)

In [449]:
# Evaluate AdaBoost Model
print("AdaBoost Accuracy:", metrics.accuracy_score(y_train, y_pred_binary))

AdaBoost Accuracy: 0.990625
